In [ ]:
from VR_Trajectory_analysis import *

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR1/20250319_symmetricBlack_angles_Data/RunData'
df1 = get_combined_df(directory, trim_seconds=1.0)

In [ ]:
directory = '/Users/apaula/ownCloud/MatrexVR1/20250327_symmetricBlack_angles_V2_Data/RunData'
df2 = get_combined_df(directory, trim_seconds=1.0)

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
df["FlyID"].unique()


In [ ]:
df = add_trial_id_and_displacement(df)
df = add_trial_time(df)

In [ ]:
df_stationary, df_normal, df_excessive, stationary_ids, normal_ids, excessive_ids = classify_trials_by_path_length(df[df['Scene']=='Choice_noBG'], min_length=0, max_length=500)

In [ ]:
plot_trajectories(df_normal, 'normal')

In [ ]:
df_normal['ConfigFile'].unique()

In [ ]:
import pandas as pd
import numpy as np
import re

def parse_angle_from_config(config_filename: str, default_angle: float = 20.0) -> float:
    """
    Extracts the 'XX' in e.g. "constantSize_XXdeg" from the config filename.
    If no match is found (deg info is missing), returns default_angle (usually 20°).
    """
    pattern = r"_([0-9]+)deg_"  # looks for something like "_30deg_"
    match = re.search(pattern, config_filename)
    if match:
        return float(match.group(1))
    else:
        return default_angle

def get_left_right_goals(angle_deg: float, radius: float = 60.0):
    """
    Given a total separation angle (in degrees) and a circle radius, 
    return (x,z) for the left goal and right goal, respectively,
    placed symmetrically about the positive z-axis.
    """
    # Convert half the separation angle to radians
    half_angle_rad = np.radians(angle_deg / 2.0)

    # By convention here:
    #  - "left" is at -half_angle_rad
    #  - "right" is at +half_angle_rad
    #
    # Since 0° is along +z, we do:
    #   x = r * sin(theta)
    #   z = r * cos(theta)
    #
    # left uses -half_angle_rad; right uses +half_angle_rad
    left_x = radius * np.sin(-half_angle_rad)
    left_z = radius * np.cos(-half_angle_rad)
    right_x = radius * np.sin(half_angle_rad)
    right_z = radius * np.cos(half_angle_rad)

    return (left_x, left_z), (right_x, right_z)

def get_first_goal_reached(
    df_normal,
    goals,            # A list of (goal_name, (x, z)) tuples
    threshold=3.5
):
    """
    Given a subset of data for a single config, determines the first 
    goal reached for each UniqueTrialID (unchanged from your snippet).
    """
    def distance(p1, p2):
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
    
    results = []
    
    # Group by UniqueTrialID
    for trial_id, trial_data in df_normal.groupby('UniqueTrialID'):
        config = trial_data['ConfigFile'].iloc[0]
        
        # Sort by time
        trial_data = trial_data.sort_values(by='trial_time')
        
        first_reached = None
        reached_time = None
        
        for idx, row in trial_data.iterrows():
            participant_pos = (row['GameObjectPosX'], row['GameObjectPosZ'])
            # Check each goal
            for goal_name, goal_pos in goals:
                if distance(participant_pos, goal_pos) <= threshold:
                    first_reached = goal_name
                    reached_time = row['trial_time']
                    break
            if first_reached is not None:
                break
        
        results.append((trial_id, config, first_reached, reached_time))
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results, columns=[
        'UniqueTrialID',
        'ConfigFile',
        'FirstReachedGoal',
        'GoalReachedTime'
    ])
    
    return results_df


def analyze_variable_angle_configs(df_normal, threshold=3.5, default_angle=20.0):
    """
    1) Finds unique config files that start with "BinaryChoice_constantSize" and 
       end with "BlackCylinder_BlackCylinder.json".
    2) Parses the angle, sets default to 20° if missing.
    3) Computes left/right goals for each angle.
    4) Calls get_first_goal_reached for each subset, concatenates results.
    """
    # Filter only the configs we care about
    mask = (
        df_normal['ConfigFile'].str.contains("BinaryChoice") &
        df_normal['ConfigFile'].str.contains("BlackCylinder_BlackCylinder")
    )
    df_subset = df_normal[mask]
    
    unique_configs = df_subset['ConfigFile'].unique()
    
    all_results = []
    
    for cfg in unique_configs:
        angle_deg = parse_angle_from_config(cfg, default_angle=default_angle)
        (left_goal, right_goal) = get_left_right_goals(angle_deg, radius=60)
        
        # Subset data for this particular config
        df_config = df_subset[df_subset['ConfigFile'] == cfg].copy()
        
        # Determine the first reached goal
        res = get_first_goal_reached(
            df_config,
            goals=[("left", left_goal), ("right", right_goal)],
            threshold=threshold
        )
        all_results.append(res)
    
    if all_results:
        return pd.concat(all_results, ignore_index=True)
    else:
        # No matching config files found
        return pd.DataFrame(columns=['UniqueTrialID', 'ConfigFile', 'FirstReachedGoal', 'GoalReachedTime'])

# -------------------------------------------------------------------
# Example Usage:
final_results = analyze_variable_angle_configs(df_normal, threshold=3.5, default_angle=20.0)
#
# final_results will have one row per UniqueTrialID, giving you
# the earliest goal reached and its time, for each of the "BinaryChoice_constantSize_..._BlackCylinder_BlackCylinder.json"
# config files in df_normal.
# -------------------------------------------------------------------


In [ ]:
get_left_right_goals(50, radius=60)

In [ ]:
final_results

In [ ]:
final_results['ConfigFile'].unique()

In [ ]:
final_results.to_pickle("symmetric_geometry_results_df.pkl")

In [ ]:
results_df = pd.read_pickle("symmetric_geometry_results_df.pkl")

In [ ]:
results_df['FirstReachedGoal'].unique()

In [ ]:
# 1. Keep only trials that actually reached a goal
valid_results = results_df.dropna(subset=['FirstReachedGoal'])


# 2. Merge the cutoff times back into df
#    We merge on 'UniqueTrialID' to get each trial's GoalReachedTime.
df_merged = pd.merge(df, valid_results[['UniqueTrialID', 'GoalReachedTime', 'FirstReachedGoal']], on='UniqueTrialID', how='inner')

# 3. Filter df so that only rows with trial_time less than or equal to the goal time are kept
df_cut = df_merged[df_merged['trial_time'] <= df_merged['GoalReachedTime']]

In [ ]:
df_cut["FlyID"].nunique()

In [ ]:
# Save DataFrames as pickle (binary, preserves dtypes & indexes)
df_cut_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/Geometry/df_cut.pkl'
results_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/Geometry/results_df.pkl'
df_cut.to_pickle(df_cut_path_pickle)
results_df.to_pickle(results_df_path_pickle)

In [ ]:
from VR_Trajectory_analysis import *
df_cut_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/Geometry/df_cut.pkl'
results_df_path_pickle = '/Users/apaula/src/VRDataAnalysis/Ants/preprocessed_data/Geometry/results_df.pkl'
df_cut = pd.read_pickle(df_cut_path_pickle)
results_df = pd.read_pickle(results_df_path_pickle)

In [ ]:
df_cut["FlyID"].nunique()

In [ ]:
def parse_angle_from_config(config_filename: str, default_angle: float = 20.0) -> float:
    """
    Extracts the 'XX' in e.g. "constantSize_XXdeg" from the config filename.
    If no match is found (deg info is missing), returns default_angle (usually 20°).
    """
    pattern = r"_([0-9]+)deg_"  # looks for something like "_30deg_"
    match = re.search(pattern, config_filename)
    if match:
        return float(match.group(1))
    else:
        return default_angle

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assume these from previous steps:
center_only_configs = [
    "BinaryChoice10_BlackCylinder_control.json",
    "BinaryChoice10_constantSize_BlackCylinder_control.json"
]

# Identify which ConfigFiles have multiple goals
# Since we know only the two listed above are single-goal configs,
# all others are multi-goal configs.
all_configs = df_cut['ConfigFile'].unique()
multi_goal_configs = [c for c in all_configs if c not in center_only_configs]

# Merge results_df to get the goal reached information into df_cut if needed
# (Not strictly necessary if we only need ratios. We can just use results_df separately.)
# But let's have a convenient DataFrame for ratio calculations.
df_joined = pd.merge(df_cut, results_df[['UniqueTrialID', 'FirstReachedGoal']], on='UniqueTrialID', how='left')

# Group the truncated data by ConfigFile
for config, group in df_joined.groupby('ConfigFile'):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot each trial’s trajectory
    for trial_id, trial_data in group.groupby('UniqueTrialID'):
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3)
    
    ax.set_ylim(-10, 70)
    ax.set_xlim(-60, 60)
    
    # Make axes equal
    ax.set_aspect('equal', adjustable='box')

    # Set plot titles and labels
  
    angle_deg = parse_angle_from_config(config)
    angle_int = int(angle_deg)
    ax.set_title(f"{angle_int}°")
    ax.set_xlabel("X Position (cm)")
    ax.set_ylabel("Z Position (cm)")
    ax.grid(False)
    
    # If it's a multi-goal config, calculate ratio left/right
    if config in multi_goal_configs:
        # Filter the trials for this config in results_df
        config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        left_count = (config_results['FirstReachedGoal'] == 'left').sum()
        right_count = (config_results['FirstReachedGoal'] == 'right').sum()
        
        # Compute ratio (e.g., left/total and right/total)
        total = left_count + right_count
        if total > 0:
            left_ratio = left_count / total
            right_ratio = right_count / total
            ratio_text = f"Left: {left_count} ({left_ratio:.2f}), Right: {right_count} ({right_ratio:.2f})"
        else:
            ratio_text = "No goals reached"
        
        # Add text box with ratio information
        ax.text(0.05, 0.95, ratio_text, transform=ax.transAxes, va='top', bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    elif config in center_only_configs:
        # Filter the trials for this config in results_df
        config_results = results_df.loc[results_df['UniqueTrialID'].isin(group['UniqueTrialID'].unique())]
        
        # Count how many reached left vs right
        count = (config_results['FirstReachedGoal'] == 'center').sum()
        # Only center goal, no ratio needed
        ax.text(0.05, 0.95, f"Center goal only: {count}", transform=ax.transAxes, va='top', bbox=dict(boxstyle="round", fc="w", ec="0.5"))
    
    
    output_dir = '/Users/apaula/Downloads'
    # Remove '.json' from config filename and prepend 'trajectories_'
    base_name = os.path.splitext(config)[0]
    filename = f"trajectories4_{base_name}.png"
    filepath = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(filepath, dpi=300)

    plt.tight_layout()
    plt.show()

In [ ]:
df_joined.columns

In [ ]:
import math
import os
import matplotlib.pyplot as plt
import pandas as pd
import re
from matplotlib.lines import Line2D

def parse_angle_from_config(config_filename: str, default_angle: float = 20.0) -> float:
    """Extract the numeric angle from '_XXdeg_' in the filename; fallback to default if missing."""
    match = re.search(r"_([0-9]+)deg_", config_filename)
    if match:
        return float(match.group(1))
    else:
        return default_angle

# Merge results if needed
df_joined = df_cut

# 1. Get all config names and parse angles
unique_configs = df_joined['ConfigFile'].unique()
config_angle_map = {cfg: parse_angle_from_config(cfg) for cfg in unique_configs}

# 2. Sort by angle
configs_sorted = sorted(unique_configs, key=lambda c: config_angle_map[c])

# 3. Exclude angles 130 and 150
configs_filtered = [cfg for cfg in configs_sorted if config_angle_map[cfg] not in {130, 150}]

# Identify multi-goal configs (everything not in center_only_configs)
multi_goal_configs = [c for c in configs_filtered if c not in center_only_configs]

# 4. Create a single figure with subplots for the filtered configs
ncols = 3
nrows = math.ceil(len(configs_filtered) / ncols)

fig, axes = plt.subplots(nrows, ncols, figsize=(60, 32), sharex=True, sharey=True)
axes = axes.flatten()  # Flatten to handle indexing more easily
fig.patch.set_alpha(0.0)

for i, config in enumerate(configs_filtered):
    ax = axes[i]
    
    # Subset data for this config
    group = df_joined[df_joined['ConfigFile'] == config]
    
    # Create goal reach lookup
    reach_map = dict(zip(group['UniqueTrialID'], group['FirstReachedGoal']))
    
    # Plot each trial’s trajectory with color by goal
    for trial_id, trial_data in group.groupby('UniqueTrialID'):
        goal = reach_map.get(trial_id, None)
        if goal == 'left':
            color = 'blue'
        elif goal == 'right':
            color = 'green'
        else:
            color = 'gray'  # fallback
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3, linewidth=2, color=color)
    
    ax.set_xlim(-60, 60)
    ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')
    
    # Title = integer angle
    angle_deg = config_angle_map[config]
    angle_int = int(angle_deg)
    ax.set_title(f"{angle_int}°")
    
    # Label axes if on left or bottom edge
    if i % ncols == 0:
        ax.set_ylabel("Z Position (cm)")
    if i >= (nrows - 1) * ncols:
        ax.set_xlabel("X Position (cm)")

# Turn off extra axes if any remain unused
for j in range(i+1, len(axes)):
    axes[j].axis('off')

# Add a shared legend
legend_elements = [
    Line2D([0], [0], color='blue',  lw=2, label='Reached Left'),
    Line2D([0], [0], color='green', lw=2, label='Reached Right'),
    Line2D([0], [0], color='gray',  lw=2, label='Unknown')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=12)

plt.tight_layout(rect=[0, 0.05, 1, 0.95])  # leave room for legend
plt.suptitle("Trajectories Colored by First Reached Goal", fontsize=20, y=0.98)

# Save and show
output_dir = '/Users/apaula/Downloads'
filename = "all_trajectories_first_6_angles_colored.png"
filepath = os.path.join(output_dir, filename)
plt.savefig(filepath, dpi=300)

plt.show()


In [ ]:
import math
import os
import matplotlib.pyplot as plt
import pandas as pd
import re

def parse_angle_from_config(config_filename: str, default_angle: float = 20.0) -> float:
    """Extract the numeric angle from '_XXdeg_' in the filename; fallback to default if missing."""
    match = re.search(r"_([0-9]+)deg_", config_filename)
    if match:
        return float(match.group(1))
    else:
        return default_angle

# Merge results if needed
df_joined = pd.merge(
    df_cut, 
    results_df[['UniqueTrialID', 'FirstReachedGoal']], 
    on='UniqueTrialID', 
    how='left'
)

# 1. Get all config names and parse angles
unique_configs = df_joined['ConfigFile'].unique()
config_angle_map = {cfg: parse_angle_from_config(cfg) for cfg in unique_configs}

# 2. Sort by angle
configs_sorted = sorted(unique_configs, key=lambda c: config_angle_map[c])

# 3. Exclude angles 130 and 150
configs_filtered = [cfg for cfg in configs_sorted if config_angle_map[cfg] not in {130, 150}]

# Identify multi-goal configs (everything not in center_only_configs)
multi_goal_configs = [c for c in configs_filtered if c not in center_only_configs]

# 4. Create a single figure with subplots for the filtered configs
ncols = 3
nrows = math.ceil(len(configs_filtered) / ncols)

fig, axes = plt.subplots(nrows, ncols, figsize=(30, 16), sharex=True, sharey=True)
axes = axes.flatten()  # Flatten to handle indexing more easily
# Remove the figure background
fig.patch.set_alpha(0.0)

for i, config in enumerate(configs_filtered):
    ax = axes[i]
    
    # Subset data for this config
    group = df_joined[df_joined['ConfigFile'] == config]
    
    # Plot each trial’s trajectory
    for trial_id, trial_data in group.groupby('UniqueTrialID'):
        ax.plot(trial_data['GameObjectPosX'], trial_data['GameObjectPosZ'], alpha=0.3, linewidth=2)
    
    ax.set_xlim(-60, 60)
    ax.set_ylim(-10, 70)
    ax.set_aspect('equal', adjustable='box')
    
    # Title = integer angle
    angle_deg = config_angle_map[config]
    angle_int = int(angle_deg)
    ax.set_title(f"{angle_int}°")
    #ax.set_facecolor("whitesmoke")  # or "whitesmoke", etc.

    
    
    # Label axes if on left or bottom edge
    if i % ncols == 0:
        ax.set_ylabel("Z Position (cm)")
    if i >= (nrows - 1) * ncols:
        ax.set_xlabel("X Position (cm)")

# Turn off extra axes if any remain unused
for j in range(i+1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()

output_dir = '/Users/apaula/Downloads'
filename = "all_trajectories_first_6_angles.png"
filepath = os.path.join(output_dir, filename)
plt.savefig(filepath, dpi=300)

plt.show()


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import math

# --------------------------------------------
# Parameters
# --------------------------------------------
n_xbins, n_zbins = 64, 64
x_range = (-60, 60)
z_range = (0, 60)  # Cut data below Z = 0
xedges = np.linspace(*x_range, n_xbins + 1)
zedges = np.linspace(*z_range, n_zbins + 1)
x_centers = 0.5 * (xedges[:-1] + xedges[1:])
z_centers = 0.5 * (zedges[:-1] + zedges[1:])

# --------------------------------------------
# Subplot setup
# --------------------------------------------
ncols = 3
nrows = math.ceil(len(configs_filtered) / ncols)
fig, axes = plt.subplots(nrows, ncols, figsize=(30, 16), sharex=True, sharey=True)
axes = axes.flatten()

# --------------------------------------------
# Loop over geometries/configs
# --------------------------------------------
for i, config in enumerate(configs_filtered):
    ax = axes[i]
    group = df_joined[df_joined['ConfigFile'] == config]

    # Step 1: Mirror data (append negative X mirrored as positive X)
    x_original = group['GameObjectPosX'].values
    z_original = group['GameObjectPosZ'].values
    x_mirrored = -x_original
    z_mirrored = z_original

    x_all = np.concatenate([x_original, x_mirrored])
    z_all = np.concatenate([z_original, z_mirrored])

    # Step 2: Filter Z >= 0
    valid_mask = z_all >= 0
    x_all = x_all[valid_mask]
    z_all = z_all[valid_mask]

    # Step 3: 2D histogram
    H, _, _ = np.histogram2d(
        x_all, z_all,
        bins=(n_xbins, n_zbins),
        range=[x_range, z_range]
    )

    # Step 4: Per-Z-bin normalization
    H_plot = H.T  # shape: (n_zbins, n_xbins), Z vertical
    for z_idx in range(H_plot.shape[0]):
        row = H_plot[z_idx]
        row_min, row_max = row.min(), row.max()
        if row_max > row_min:
            H_plot[z_idx] = (row - row_min) / (row_max - row_min)

    # Step 5: Gaussian blur
    H_blurred = gaussian_filter(H_plot, sigma=1.0, mode='nearest')

    # Step 6: Plot heatmap
    ax.imshow(
        H_blurred,
        origin='lower',
        aspect='auto',
        cmap='viridis',
        extent=[xedges[0], xedges[-1], zedges[0], zedges[-1]]
    )

    angle_deg = config_angle_map[config]
    ax.set_title(f"{int(angle_deg)}°")
    ax.set_xlim(-60, 60)
    ax.set_ylim(0, 60)
    ax.set_aspect('equal')

    if i % ncols == 0:
        ax.set_ylabel("Z Position (cm)")
    if i >= (nrows - 1) * ncols:
        ax.set_xlabel("X Position (cm)")

# Hide unused axes
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.savefig("normalized_blurred_heatmaps.png", dpi=300)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
from scipy.ndimage import gaussian_filter
import math

# ----------------------------
# Utility: compute bifurcation goal positions
# ----------------------------
def get_left_right_goals(angle_deg: float, radius: float = 60.0):
    half_angle_rad = np.radians(angle_deg / 2.0)
    left_x = radius * np.sin(-half_angle_rad)
    left_z = radius * np.cos(-half_angle_rad)
    right_x = radius * np.sin(half_angle_rad)
    right_z = radius * np.cos(half_angle_rad)
    return (left_x, left_z), (right_x, right_z)

# ----------------------------
# Parameters
# ----------------------------
n_xbins, n_zbins = 64, 64
x_range = (-60, 60)
z_range = (0, 60)
xedges = np.linspace(*x_range, n_xbins + 1)
zedges = np.linspace(*z_range, n_zbins + 1)
x_centers = 0.5 * (xedges[:-1] + xedges[1:])
z_centers = 0.5 * (zedges[:-1] + zedges[1:])

# ----------------------------
# Subplots
# ----------------------------
ncols = 3
nrows = math.ceil(len(configs_filtered) / ncols)
fig, axes = plt.subplots(nrows, ncols, figsize=(30, 16), sharex=True, sharey=True)
axes = axes.flatten()

# ----------------------------
# Loop over configs
# ----------------------------
for i, config in enumerate(configs_filtered):
    ax = axes[i]
    group = df_joined[df_joined['ConfigFile'] == config]
    angle_deg = config_angle_map[config]
    _, (right_x, right_z) = get_left_right_goals(angle_deg)

    # Step 1: Mirror data
    x_original = group['GameObjectPosX'].values
    z_original = group['GameObjectPosZ'].values
    x_mirrored = -x_original
    z_mirrored = z_original

    x_all = np.concatenate([x_original, x_mirrored])
    z_all = np.concatenate([z_original, z_mirrored])

    # Step 2: Filter to 0 <= Z <= right_z
    valid_mask = (z_all >= 0) & (z_all <= right_z)
    x_all = x_all[valid_mask]
    z_all = z_all[valid_mask]


    # Step 3: 2D histogram
    H, _, _ = np.histogram2d(x_all, z_all, bins=(n_xbins, n_zbins), range=[x_range, z_range])

    # Step 4: Normalize per Z-bin
    H_plot = H.T
    for z_idx in range(H_plot.shape[0]):
        row = H_plot[z_idx]
        if row.max() > row.min():
            H_plot[z_idx] = (row - row.min()) / (row.max() - row.min())

    # Step 5: Gaussian blur
    H_blurred = gaussian_filter(H_plot, sigma=1.0, mode='nearest')

    # Step 6: Extract X-mode (peak)
    x_peakline = []
    for z_idx in range(H_blurred.shape[0]):
        row = H_blurred[z_idx]
        pos_mask = x_centers >= 0
        row_pos = row[pos_mask]
        x_pos = x_centers[pos_mask]
        if row_pos.sum() == 0:
            x_peakline.append(np.nan)
        else:
            x_peakline.append(x_pos[np.argmax(row_pos)])
    x_peakline = np.array(x_peakline)

    # Step 7: Fit bifurcation model (a = 0)
    z_mask = z_centers <= 57
    z_fit = z_centers[z_mask]
    x_fit = x_peakline[z_mask]
    z_candidates = np.linspace(0, 55, 200)

    best_zc = None
    best_error = np.inf
    for z_c in z_candidates:
        slope = right_x / (right_z - z_c)
        model = np.array([
            slope * (z - z_c) if z >= z_c else 0 for z in z_fit
        ])
        valid = ~np.isnan(x_fit)
        error = np.mean((x_fit[valid] - model[valid]) ** 2)
        if error < best_error:
            best_error = error
            best_zc = z_c

    # Final slope with best z_c
    slope = right_x / (right_z - best_zc)

    # Step 8: Rebuild full left/right models
    full_model_right = np.array([
        slope * (z - best_zc) if z >= best_zc else 0 for z in z_centers
    ])
    full_model_left = -full_model_right

    # Step 9: Plot heatmap
    ax.imshow(H_blurred, origin='lower', aspect='auto', cmap='viridis',
              extent=[xedges[0], xedges[-1], zedges[0], zedges[-1]])

    # Step 10: Overlay bifurcation arms
    ax.plot(full_model_right, z_centers, color='red', linewidth=2)
    ax.plot(full_model_left, z_centers, color='red', linewidth=2)

    # Step 11: Plot angle arc
    angle_rad = np.arctan(slope)
    angle_deg_fit = np.degrees(angle_rad)
    bifurcation_angle = 2 * angle_deg_fit
    arc_radius = 5
    arc = Arc((0, best_zc), 2 * arc_radius, 2 * arc_radius,
              angle=0, theta1=90 - angle_deg_fit, theta2=90 + angle_deg_fit,
              color='white', lw=2)
    ax.add_patch(arc)
    ax.text(0, best_zc + arc_radius + 0.5, f"{bifurcation_angle:.1f}°",
            ha='center', va='bottom', color='white', fontsize=10, fontweight='bold')

    # Step 12: Styling
    ax.set_xlim(-60, 60)
    ax.set_ylim(0, 60)
    ax.set_aspect('equal')
    ax.set_title(f"{int(angle_deg)}°\nzc={best_zc:.2f}, angle={bifurcation_angle:.1f}°")

    if i % ncols == 0:
        ax.set_ylabel("Z Position (cm)")
    if i >= (nrows - 1) * ncols:
        ax.set_xlabel("X Position (cm)")

# Hide unused subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.savefig("bifurcation_fits_complete.png", dpi=300)
plt.show()


In [ ]:
import numpy as np
import pandas as pd
df = df_cut.copy()

# --- 1.  Make sure the rows are ordered inside each trial ---------------
df = df.sort_values(['UniqueTrialID', 'trial_time'])

# --- 2.  Per-trial differences -----------------------------------------
# Δx and Δz (horizontal plane)
df['dx'] = df.groupby('UniqueTrialID')['GameObjectPosX'].diff()
df['dz'] = df.groupby('UniqueTrialID')['GameObjectPosZ'].diff()

# Δt – time step in seconds
df['dt'] = df.groupby('UniqueTrialID')['trial_time'].diff()

# --- 3.  Instantaneous speed (magnitude, m / s) ------------------------
df['speed_inst'] = np.sqrt(df['dx']**2 + df['dz']**2) / df['dt']

# Optionally scrub rows where dt==0 or <0 (if any jitter slipped in)
df.loc[df['dt'] <= 0, 'speed_inst'] = np.nan

# --- 4.  Mean speed per trial (one number each) ------------------------
trial_speed = (df.groupby('UniqueTrialID')
                 .apply(lambda g: np.nanmean(g['speed_inst']))
                 .rename('speed_mean')
                 .reset_index())

# trial_speed is a tidy two-column DataFrame:
#   UniqueTrialID   speed_mean


In [ ]:
df['UniqueTrialID'].nunique()

In [ ]:
example_trial = (df['UniqueTrialID'].unique()[12]
)

trial_df = df[df['UniqueTrialID'] == example_trial].copy()


In [ ]:
example_trial = (df['UniqueTrialID'].unique()[1]
)

trial_df = df[df['UniqueTrialID'] == example_trial].copy()
# 500 ms window as an example — adjust to taste
win_seconds = 0.3
# figure out how many rows that is per trial using the median dt
rows_per_second = 1 / trial_df['dt'].median()
win = int(win_seconds * rows_per_second)

trial_df['speed_smooth'] = (
    trial_df['speed_inst']
    .rolling(win, center=True, min_periods=1)
    .mean()
)
plt.figure(figsize=(14,10))
plt.plot(trial_df['trial_time'], trial_df['speed_inst'], label='raw', lw=0.8, alpha=0.6)
plt.plot(trial_df['trial_time'], trial_df['speed_smooth'], label='rolling mean', lw=2)
plt.ylim(0,10)
plt.xlim(3,10)
plt.xlabel('Trial time [s]')
plt.ylabel('Speed [cm s⁻¹]')
plt.title(f'Speed profile – trial {example_trial}')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
trial_df.columns

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(trial_df['trial_time'], trial_df['speed_inst'], label='raw', lw=0.8, alpha=0.6)
plt.plot(trial_df['trial_time'], trial_df['speed_smooth'], label='rolling mean', lw=2)
plt.ylim(0,10)
plt.xlim(3,10)
plt.xlabel('Trial time [s]')
plt.ylabel('Speed [cm s⁻¹]')
plt.title(f'Speed profile – trial {example_trial}')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 500 ms window as an example — adjust to taste
win_seconds = 0.3
# figure out how many rows that is per trial using the median dt
rows_per_second = 1 / df['dt'].median()
win = int(win_seconds * rows_per_second)

df['speed_smooth'] = (
    df['speed_inst']
    .rolling(win, center=True, min_periods=1)
    .mean()
)


In [ ]:
import re, math, numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
# ------------- legend handles (create once) -----------------
goal_line        = mlines.Line2D([], [], color='red',   ls='--', lw=1.4,
                                label='goal position')
bifurcation_line = mlines.Line2D([], [], color='black', ls=':',  lw=1.5,
                                label='fitted bifurcation')

extra_lines = {0: 49.75,   # subplot 0 (col 0, row 0) → x = 50
        1: 41.62,   # subplot 1 (col 1, row 0) → x = 45
        2: 19.62,   # subplot 2 (col 2, row 0) → x = 25
        3: 5.53,
        4: 1.93}   # subplot 3 (col 3, row 0) → x = 10
# -------------------------------------------------------------
# 0.  Collect configs → sort by angle (ascending)
# -------------------------------------------------------------
cfg_angles = [(cfg, parse_angle_from_config(cfg)) 
              for cfg in df['ConfigFile'].unique()]

configs_sorted = [cfg for cfg, _ in sorted(cfg_angles, key=lambda t: t[1])]

configs_sorted = configs_sorted[:6]  # Limit to first 6
n_cfg  = len(configs_sorted)           # 8 expected
n_cols = 3
n_rows = math.ceil(n_cfg / n_cols)

# common y-limit (95-th pct keeps a few spikes from stretching scale)
y_max = df['speed_smooth'].quantile(0.95)

# constants
BAND_HALFWIDTH = 3.5
RADIUS         = 60.0
XLIM           = (0, 60)

# -------------------------------------------------------------
# 1.  Helper to draw a single panel
# -------------------------------------------------------------
def plot_profile(ax, df_cfg, cfg_name):
    # goal Z
    angle_deg = parse_angle_from_config(cfg_name)
    (_, left_z), _ = get_left_right_goals(angle_deg, radius=RADIUS)

    # summary in 100 Z-bins
    bins = np.linspace(XLIM[0], XLIM[1], 51)
    df_cfg = df_cfg[(df_cfg['GameObjectPosZ'] >= XLIM[0]) &
                    (df_cfg['GameObjectPosZ'] <= XLIM[1])].copy()
    df_cfg['Z_bin'] = np.clip(np.digitize(df_cfg['GameObjectPosZ'], bins) - 1, 0, 99)

    summary = (df_cfg.groupby('Z_bin')['speed_smooth']
                 .agg(mean='mean', std='std')
                 .reset_index())
    summary['Z_mid'] = summary['Z_bin'].map(lambda i: 0.5*(bins[i] + bins[i+1]))

    # plot
    ax.plot(summary['Z_mid'], summary['mean'], lw=2)
    ax.fill_between(summary['Z_mid'],
                    summary['mean'] - summary['std'],
                    summary['mean'] + summary['std'],
                    alpha=0.25)

    # goal marker & band
    ax.axvline(left_z, color='red', ls='--', lw=1.2)
    ax.axvspan(left_z - BAND_HALFWIDTH, left_z + BAND_HALFWIDTH,
               color='red', alpha=0.15)

    for idx, cfg in enumerate(configs_sorted):
        r, c = divmod(idx, n_cols)
        ax   = axes[r, c] if n_rows > 1 else axes[c]

        # …everything we already did to build summary, plot mean ± sd, goal band…

        # >>> NEW: dotted black line in the first four panels
        if idx in extra_lines:
            ax.axvline(extra_lines[idx],
                    color='black', ls=':', lw=1.5,
                    label=f'x = {extra_lines[idx]}')
    ax.set_xlim(*XLIM)
    ax.set_ylim(0, y_max)
    ax.tick_params(labelsize=8)

# -------------------------------------------------------------
# 2.  Build the grid
# -------------------------------------------------------------
fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(n_cols*4, n_rows*3),
                         sharey=True)

for idx, cfg in enumerate(configs_sorted):
    r, c = divmod(idx, n_cols)
    ax   = axes[r, c] if n_rows > 1 else axes[c]

    df_cfg = (df[df['ConfigFile'] == cfg]
              .sort_values(['UniqueTrialID', 'trial_time']))
    angle_deg = parse_angle_from_config(cfg)  # <- moved here
    plot_profile(ax, df_cfg, cfg)
    ax.set_title(f"{int(angle_deg)}°", fontsize=10)  # <- add title here

# hide unused squares if any
for ax in axes.flat[n_cfg:]:
    ax.set_visible(False)

# outer labels
fig.text(0.5, 0.04, 'GameObject Z position', ha='center', fontsize=11)
fig.text(0.04, 0.5, 'Smoothed speed (m s⁻¹)', va='center',
         rotation='vertical', fontsize=11)
fig.suptitle('Average speed vs Z \nConfigs sorted by angle',
             y=0.98, fontsize=13)
fig.tight_layout(rect=[0.05, 0.05, 0.99, 0.95])
# ------------- add a single legend to the whole figure ---------------
fig.legend(handles=[bifurcation_line, goal_line],
           loc='upper right', bbox_to_anchor=(0.99, 0.99),
           frameon=False, fontsize=10)
plt.show()


In [ ]:
import re, math, numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
# ------------- legend handles (create once) -----------------
goal_line        = mlines.Line2D([], [], color='red',   ls='--', lw=1.4,
                                label='goal position')
bifurcation_line = mlines.Line2D([], [], color='black', ls=':',  lw=1.5,
                                label='fitted bifurcation')

extra_lines = {0: 49.75,   # subplot 0 (col 0, row 0) → x = 50
        1: 41.62,   # subplot 1 (col 1, row 0) → x = 45
        2: 19.62,   # subplot 2 (col 2, row 0) → x = 25
        3: 5.53,
        4: 1.93}   # subplot 3 (col 3, row 0) → x = 10
# -------------------------------------------------------------
# 0.  Collect configs → sort by angle (ascending)
# -------------------------------------------------------------
cfg_angles = [(cfg, parse_angle_from_config(cfg)) 
              for cfg in df['ConfigFile'].unique()]

configs_sorted = [cfg for cfg, _ in sorted(cfg_angles, key=lambda t: t[1])]

configs_sorted = configs_sorted[:6]  # Limit to first 6
n_cfg  = len(configs_sorted)           # 8 expected
n_cols = 3
n_rows = math.ceil(n_cfg / n_cols)

# common y-limit (95-th pct keeps a few spikes from stretching scale)
y_max = df['speed_smooth'].quantile(0.95)

# constants
BAND_HALFWIDTH = 3.5
RADIUS         = 60.0
XLIM           = (0, 60)

# -------------------------------------------------------------
# 1.  Helper to draw a single panel
# -------------------------------------------------------------
def plot_profile(ax, df_cfg, cfg_name):
    # goal Z
    angle_deg = parse_angle_from_config(cfg_name)
    (_, left_z), _ = get_left_right_goals(angle_deg, radius=RADIUS)

    # summary in 100 Z-bins
    bins = np.linspace(XLIM[0], XLIM[1], 51)
    df_cfg = df_cfg[(df_cfg['GameObjectPosZ'] >= XLIM[0]) &
                    (df_cfg['GameObjectPosZ'] <= XLIM[1])].copy()
    df_cfg['Z_bin'] = np.clip(np.digitize(df_cfg['GameObjectPosZ'], bins) - 1, 0, 99)

    summary = (df_cfg.groupby('Z_bin')['speed_smooth']
                .agg(median=lambda x: np.nanmedian(np.abs(x)),
    q25=lambda x: np.nanpercentile(np.abs(x), 25),
    q75=lambda x: np.nanpercentile(np.abs(x), 75)
    )

                 .reset_index())
    summary['Z_mid'] = summary['Z_bin'].map(lambda i: 0.5*(bins[i] + bins[i+1]))

    # plot
    ax.plot(summary['Z_mid'], summary['median'], lw=2)
    ax.fill_between(summary['Z_mid'],
                    summary['q25'],
                    summary['q75'],
                    alpha=0.25)


    # goal marker & band
    ax.axvline(left_z, color='red', ls='--', lw=1.2)
    ax.axvspan(left_z - BAND_HALFWIDTH, left_z + BAND_HALFWIDTH,
               color='red', alpha=0.15)

    for idx, cfg in enumerate(configs_sorted):
        r, c = divmod(idx, n_cols)
        ax   = axes[r, c] if n_rows > 1 else axes[c]

        # …everything we already did to build summary, plot mean ± sd, goal band…

        # >>> NEW: dotted black line in the first four panels
        if idx in extra_lines:
            ax.axvline(extra_lines[idx],
                    color='black', ls=':', lw=1.5,
                    label=f'x = {extra_lines[idx]}')
    ax.set_xlim(*XLIM)
    ax.set_ylim(0, y_max)
    ax.set_title(f"{angle_deg:.0f}° – {cfg_name.split('/')[-1]}", fontsize=9)
    ax.tick_params(labelsize=8)

# -------------------------------------------------------------
# 2.  Build the grid
# -------------------------------------------------------------
fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(n_cols*4, n_rows*3),
                         sharey=True)

for idx, cfg in enumerate(configs_sorted):
    r, c = divmod(idx, n_cols)
    ax   = axes[r, c] if n_rows > 1 else axes[c]

    df_cfg = (df[df['ConfigFile'] == cfg]
              .sort_values(['UniqueTrialID', 'trial_time']))

    plot_profile(ax, df_cfg, cfg)

# hide unused squares if any
for ax in axes.flat[n_cfg:]:
    ax.set_visible(False)

# outer labels
fig.text(0.5, 0.04, 'GameObject Z position', ha='center', fontsize=11)
fig.text(0.04, 0.5, 'Smoothed speed (m s⁻¹)', va='center',
         rotation='vertical', fontsize=11)
fig.suptitle('Average speed vs Z (0.3 s smoothing; ±3.5 band)\nConfigs sorted by angle',
             y=0.98, fontsize=13)
fig.tight_layout(rect=[0.05, 0.05, 0.99, 0.95])
# ------------- add a single legend to the whole figure ---------------
fig.legend(handles=[bifurcation_line, goal_line],
           loc='upper right', bbox_to_anchor=(0.99, 0.99),
           frameon=False, fontsize=10)
plt.show()


In [ ]:
# --- 5. Turning rate (from GameObjectRotY) ----------------------

# Unwrap rotation (convert degrees to radians first)
df['rot_y_rad'] = np.radians(df['GameObjectRotY'])
df['rot_y_unwrapped'] = (
    df.groupby('UniqueTrialID')['rot_y_rad']
    .transform(np.unwrap)
)

# Compute turning rate (rad/s), then convert to deg/s
df['dtheta_rad'] = df.groupby('UniqueTrialID')['rot_y_unwrapped'].diff()
df['turning_rate'] = np.degrees(df['dtheta_rad']) / df['dt']
# Smoothed turning rate
df['turning_smooth'] = (
    df['turning_rate']
    .rolling(17, center=True, min_periods=1)
    .mean()
)

# Optional cleanup
df.loc[df['dt'] <= 0, 'turning_rate'] = np.nan


In [ ]:
rows_per_second = 1 / df['dt'].median()
int(win_seconds * rows_per_second)

In [ ]:
example_trial = (df['UniqueTrialID'].unique()[30])
                 
                 # Use same example trial
trial_df = df[df['UniqueTrialID'] == example_trial].copy()

win_seconds = 0.3
# Compute smoothing window (same logic as before)
rows_per_second = 1 / trial_df['dt'].median()
win = int(win_seconds * rows_per_second)

# Smoothed turning rate
trial_df['turning_smooth'] = (
    trial_df['turning_rate']
    .rolling(17, center=True, min_periods=1)
    .mean()
)

# Plot
plt.figure(figsize=(14, 6))
plt.plot(trial_df['trial_time'], trial_df['turning_rate'], label='raw', lw=0.8, alpha=0.6)
plt.plot(trial_df['trial_time'], trial_df['turning_smooth'], label='rolling mean', lw=2)
plt.axhline(0, color='gray', linestyle='--', lw=0.8)

plt.xlabel('Trial time [s]')
plt.ylabel('Turning rate [°/s]')
plt.title(f'Turning rate – trial {example_trial}')
plt.xlim(3, 10)
plt.ylim(-200, 200)  # Adjust based on your range
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import re, math, numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

# ------------- legend handles -----------------
goal_line = mlines.Line2D([], [], color='red', ls='--', lw=1.4,
                          label='goal position')
bifurcation_line = mlines.Line2D([], [], color='black', ls=':', lw=1.5,
                                 label='fitted bifurcation')

extra_lines = {
    0: 49.75,
    1: 41.62,
    2: 19.62,
    3: 5.53,
    4: 1.93
}

# -------------------------------------------------------------
# 0.  Collect configs → sort by angle (ascending)
# -------------------------------------------------------------
cfg_angles = [(cfg, parse_angle_from_config(cfg)) for cfg in df['ConfigFile'].unique()]
configs_sorted = [cfg for cfg, _ in sorted(cfg_angles, key=lambda t: t[1])]
configs_sorted = configs_sorted[:6]  # Limit to first 6

n_cfg = len(configs_sorted)
n_cols = 3
n_rows = math.ceil(n_cfg / n_cols)

# common y-limit for turning rate (75th percentile of abs values)
y_max = df['turning_smooth'].abs().quantile(0.95)

# constants
BAND_HALFWIDTH = 3.5
RADIUS = 60.0
XLIM = (0, 60)

# -------------------------------------------------------------
# 1.  Helper to draw a single panel
# -------------------------------------------------------------
def plot_profile(ax, df_cfg, cfg_name, idx):
    angle_deg = parse_angle_from_config(cfg_name)
    (_, left_z), _ = get_left_right_goals(angle_deg, radius=RADIUS)

    # Bin along Z
    bins = np.linspace(XLIM[0], XLIM[1], 30)
    df_cfg = df_cfg[(df_cfg['GameObjectPosZ'] >= XLIM[0]) &
                    (df_cfg['GameObjectPosZ'] <= XLIM[1])].copy()
    df_cfg['Z_bin'] = np.clip(np.digitize(df_cfg['GameObjectPosZ'], bins) - 1, 0, 99)

    # Compute median |turning rate| and spread (std)
    summary = df_cfg.groupby('Z_bin')['turning_smooth'].agg(
        median=lambda x: np.nanmedian(np.abs(x)),
        q25=lambda x: np.nanpercentile(np.abs(x), 25),
        q75=lambda x: np.nanpercentile(np.abs(x), 75)
    ).reset_index()


    summary['Z_mid'] = summary['Z_bin'].map(lambda i: 0.5 * (bins[i] + bins[i+1]))

    # Plot median ± std
    ax.plot(summary['Z_mid'], summary['median'], lw=2)
    ax.fill_between(summary['Z_mid'],
                    summary['q25'],
                    summary['q75'],
                    alpha=0.25)


    # Goal marker & shaded region
    ax.axvline(left_z, color='red', ls='--', lw=1.2)
    ax.axvspan(left_z - BAND_HALFWIDTH, left_z + BAND_HALFWIDTH,
               color='red', alpha=0.15)

    # Optional: bifurcation lines
    if idx in extra_lines:
        ax.axvline(extra_lines[idx], color='black', ls=':', lw=1.5)

    ax.set_xlim(*XLIM)
    ax.set_ylim(0, y_max)
    ax.set_title(f"{int(angle_deg)}°", fontsize=10)
    ax.tick_params(labelsize=8)

# -------------------------------------------------------------
# 2.  Build the grid
# -------------------------------------------------------------
fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(n_cols * 4, n_rows * 3),
                         sharey=True)

for idx, cfg in enumerate(configs_sorted):
    r, c = divmod(idx, n_cols)
    ax = axes[r, c] if n_rows > 1 else axes[c]

    df_cfg = df[df['ConfigFile'] == cfg].sort_values(['UniqueTrialID', 'trial_time'])
    plot_profile(ax, df_cfg, cfg, idx)

# Hide unused subplots
for ax in axes.flat[n_cfg:]:
    ax.set_visible(False)

# Outer labels
fig.text(0.5, 0.04, 'GameObject Z position (cm)', ha='center', fontsize=11)
fig.text(0.04, 0.5, 'Median |turning rate| (°/s)', va='center',
         rotation='vertical', fontsize=11)

fig.suptitle('Median absolute turning rate vs Z\nConfigs sorted by angle',
             y=0.98, fontsize=13)
fig.tight_layout(rect=[0.05, 0.05, 0.99, 0.95])

# Legend
fig.legend(handles=[bifurcation_line, goal_line],
           loc='upper right', bbox_to_anchor=(0.99, 0.99),
           frameon=False, fontsize=10)

plt.show()


In [ ]:
import re, math, numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

# --- Legend handles ---
goal_line = mlines.Line2D([], [], color='red', ls='--', lw=1.4, label='goal position')
bifurcation_line = mlines.Line2D([], [], color='black', ls=':', lw=1.5, label='fitted bifurcation')

extra_lines = {
    0: 49.75,
    1: 41.62,
    2: 19.62,
    3: 5.53,
    4: 1.93
}

# --- Configs sorted by angle ---
cfg_angles = [(cfg, parse_angle_from_config(cfg)) for cfg in df['ConfigFile'].unique()]
configs_sorted = [cfg for cfg, _ in sorted(cfg_angles, key=lambda t: t[1])][:6]

n_cfg = len(configs_sorted)
n_cols = 3
n_rows = math.ceil(n_cfg / n_cols)

# --- Constants ---
BAND_HALFWIDTH = 3.5
RADIUS = 60.0
XLIM = (0, 60)
N_BINS = 30  # <- Number of quantile bins

# -------------------------------------------------------------
# Universal plotting function (median ± IQR with quantile bins)
# -------------------------------------------------------------
def plot_profiles(variable: str, ylabel: str, title: str, transform=np.abs):
    y_max = transform(df[variable]).quantile(0.95)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 3), sharey=True)

    for idx, cfg in enumerate(configs_sorted):
        r, c = divmod(idx, n_cols)
        ax = axes[r, c] if n_rows > 1 else axes[c]

        df_cfg = df[df['ConfigFile'] == cfg].sort_values(['UniqueTrialID', 'trial_time']).copy()
        angle_deg = parse_angle_from_config(cfg)
        (_, left_z), _ = get_left_right_goals(angle_deg, radius=RADIUS)

        # Filter Z range
        df_cfg = df_cfg[(df_cfg['GameObjectPosZ'] >= XLIM[0]) &
                        (df_cfg['GameObjectPosZ'] <= XLIM[1])]

        # Quantile binning based on Z
        df_cfg['Z_bin'], bin_edges = pd.qcut(
            df_cfg['GameObjectPosZ'],
            q=N_BINS,
            labels=False,
            retbins=True,
            duplicates='drop'
        )
        z_mids = 0.5 * (bin_edges[:-1] + bin_edges[1:])

        # Summary stats: median + IQR of transformed variable
        summary = df_cfg.groupby('Z_bin')[variable].agg(
            median=lambda x: np.nanmedian(transform(x)),
            q25=lambda x: np.nanpercentile(transform(x), 25),
            q75=lambda x: np.nanpercentile(transform(x), 75)
        ).reset_index()

        summary['Z_mid'] = summary['Z_bin'].map(lambda i: z_mids[int(i)])

        # Plot
        ax.plot(summary['Z_mid'], summary['median'], lw=2)
        ax.fill_between(summary['Z_mid'], summary['q25'], summary['q75'], alpha=0.25)

        # Goal marker & band
        ax.axvline(left_z, color='red', ls='--', lw=1.2)
        ax.axvspan(left_z - BAND_HALFWIDTH, left_z + BAND_HALFWIDTH, color='red', alpha=0.15)

        if idx in extra_lines:
            ax.axvline(extra_lines[idx], color='black', ls=':', lw=1.5)

        ax.set_xlim(*XLIM)
        ax.set_ylim(0, y_max)
        ax.set_title(f"{int(angle_deg)}°", fontsize=10)
        ax.tick_params(labelsize=8)

    # Hide unused subplots
    for ax in axes.flat[n_cfg:]:
        ax.set_visible(False)

    # Labels
    fig.text(0.5, 0.04, 'GameObject Z position (cm)', ha='center', fontsize=11)
    fig.text(0.04, 0.5, ylabel, va='center', rotation='vertical', fontsize=11)
    fig.suptitle(title, y=0.98, fontsize=13)
    fig.tight_layout(rect=[0.05, 0.05, 0.99, 0.95])

    # Legend
    fig.legend(handles=[bifurcation_line, goal_line],
               loc='upper right', bbox_to_anchor=(0.99, 0.99),
               frameon=False, fontsize=10)

    plt.show()


In [ ]:
plot_profiles(
    variable='speed_smooth',
    ylabel='Smoothed speed (cm/s)',
    title='Median smoothed speed vs Z (equal-size Z bins)'
)


In [ ]:
plot_profiles(
    variable='turning_smooth',
    ylabel='|Turning rate| (°/s)',
    title='Median absolute turning rate vs Z (equal-size Z bins)'
)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------------
# 0   Sanity-check: the smoothed speed column must exist
# ------------------------------------------------------------------
assert 'speed_smooth' in df.columns, "Run the smoothing step first!"

# ------------------------------------------------------------------
# 1   Trial-level mean speed (one row per UniqueTrialID)
# ------------------------------------------------------------------
trial_mean = (
    df.groupby('UniqueTrialID')
      .agg(mean_speed=('speed_smooth', 'mean'),
           FlyID      =('FlyID',       'first'),
           VR         =('VR',          'first'))
      .reset_index()
)

# ------------------------------------------------------------------
# 2   Helper: mean ± 95 % CI *across trials* for any grouping key
# ------------------------------------------------------------------
def mean_ci(trial_df, key):
    out = (trial_df.groupby(key)['mean_speed']
             .agg(mean='mean', std='std', n='size')
             .reset_index())
    out['sem']  = out['std'] / np.sqrt(out['n'])
    out['ci95'] = 1.96 * out['sem']          # 95 % CI
    return out

fly_stats = mean_ci(trial_mean, 'FlyID')
vr_stats  = mean_ci(trial_mean, 'VR')

display(fly_stats.head(), vr_stats.head())    # quick look

# ------------------------------------------------------------------
# 3   Bar plots with 95 % CI – now each bar is “average of trials”
# ------------------------------------------------------------------
def bar_with_ci(stats, x_col, title, figsize=(10,5)):
    stats_sorted = stats.sort_values('mean')
    x     = stats_sorted[x_col].astype(str)
    y     = stats_sorted['mean']
    err   = stats_sorted['ci95']
    
    plt.figure(figsize=figsize)
    plt.bar(x, y, yerr=err, capsize=4)
    plt.ylabel("Mean smoothed speed (m s⁻¹)")
    plt.title(title)
    plt.xticks(rotation=60, ha='right')
    
    # annotate n per bar if useful
    for xi, yi, ni in zip(range(len(x)), y, stats_sorted['n']):
        plt.text(xi, yi + err.iloc[xi] + 0.03, f"n={ni}", ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()

bar_with_ci(fly_stats, 'FlyID', "Mean trial-speed per Fly (±95 % CI)")
bar_with_ci(vr_stats,  'VR',    "Mean trial-speed per VR (±95 % CI)")


In [ ]:
# 1) trial-level mean of the smoothed speed --------------------------
trial_baseline = (
    df.groupby('UniqueTrialID')['speed_smooth']
      .mean()
      .rename('trial_mean_speed')
)

# 2) merge the baseline onto every row -------------------------------
df = df.merge(trial_baseline, on='UniqueTrialID', how='left')

# 3) normalise each row’s speed --------------------------------------
df['speed_norm'] = df['speed_smooth'] / df['trial_mean_speed']



In [ ]:
print(df[['FlyID','speed_norm']].groupby('FlyID').mean().head())
# should be ~1.0 for every fly (numerical noise is fine)

print(df['speed_norm'].describe())
# overall distribution centred at 1, spread shows within-fly variability


In [ ]:
import re, math, numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
# ------------- legend handles (create once) -----------------
goal_line        = mlines.Line2D([], [], color='red',   ls='--', lw=1.4,
                                label='goal position')
bifurcation_line = mlines.Line2D([], [], color='black', ls=':',  lw=1.5,
                                label='approx. bifurcation')

extra_lines = {0: 48,   # subplot 0 (col 0, row 0) → x = 50
        1: 42,   # subplot 1 (col 1, row 0) → x = 45
        2: 25,   # subplot 2 (col 2, row 0) → x = 25
        3: 10}   # subplot 3 (col 3, row 0) → x = 10
# -------------------------------------------------------------
# 0.  Collect configs → sort by angle (ascending)
# -------------------------------------------------------------
cfg_angles = [(cfg, parse_angle_from_config(cfg)) 
              for cfg in df['ConfigFile'].unique()]

configs_sorted = [cfg for cfg, _ in sorted(cfg_angles, key=lambda t: t[1])]

n_cfg  = len(configs_sorted)           # 8 expected
n_cols = 4
n_rows = math.ceil(n_cfg / n_cols)

# common y-limit (95-th pct keeps a few spikes from stretching scale)
y_max = df['speed_norm'].quantile(0.95)

# constants
BAND_HALFWIDTH = 3.5
RADIUS         = 60.0
XLIM           = (0, 60)

# -------------------------------------------------------------
# 1.  Helper to draw a single panel
# -------------------------------------------------------------
def plot_profile(ax, df_cfg, cfg_name):
    # goal Z
    angle_deg = parse_angle_from_config(cfg_name)
    (_, left_z), _ = get_left_right_goals(angle_deg, radius=RADIUS)

    # summary in 100 Z-bins
    bins = np.linspace(XLIM[0], XLIM[1], 101)
    df_cfg = df_cfg[(df_cfg['GameObjectPosZ'] >= XLIM[0]) &
                    (df_cfg['GameObjectPosZ'] <= XLIM[1])].copy()
    df_cfg['Z_bin'] = np.clip(np.digitize(df_cfg['GameObjectPosZ'], bins) - 1, 0, 99)

    summary = (df_cfg.groupby('Z_bin')['speed_norm']
                 .agg(mean='mean', std='std')
                 .reset_index())
    summary['Z_mid'] = summary['Z_bin'].map(lambda i: 0.5*(bins[i] + bins[i+1]))

    # plot
    ax.plot(summary['Z_mid'], summary['mean'], lw=2)
    ax.fill_between(summary['Z_mid'],
                    summary['mean'] - summary['std'],
                    summary['mean'] + summary['std'],
                    alpha=0.25)

    # goal marker & band
    ax.axvline(left_z, color='red', ls='--', lw=1.2)
    ax.axvspan(left_z - BAND_HALFWIDTH, left_z + BAND_HALFWIDTH,
               color='red', alpha=0.15)

    for idx, cfg in enumerate(configs_sorted):
        r, c = divmod(idx, n_cols)
        ax   = axes[r, c] if n_rows > 1 else axes[c]

        # …everything we already did to build summary, plot mean ± sd, goal band…

        # >>> NEW: dotted black line in the first four panels
        if idx in extra_lines:
            ax.axvline(extra_lines[idx],
                    color='black', ls=':', lw=1.5,
                    label=f'x = {extra_lines[idx]}')
    ax.set_xlim(*XLIM)
    ax.set_ylim(0, y_max)
    ax.set_title(f"{angle_deg:.0f}° – {cfg_name.split('/')[-1]}", fontsize=9)
    ax.tick_params(labelsize=8)

# -------------------------------------------------------------
# 2.  Build the grid
# -------------------------------------------------------------
fig, axes = plt.subplots(n_rows, n_cols,
                         figsize=(n_cols*4, n_rows*3),
                         sharey=True)

for idx, cfg in enumerate(configs_sorted):
    r, c = divmod(idx, n_cols)
    ax   = axes[r, c] if n_rows > 1 else axes[c]

    df_cfg = (df[df['ConfigFile'] == cfg]
              .sort_values(['UniqueTrialID', 'trial_time']))

    plot_profile(ax, df_cfg, cfg)

# hide unused squares if any
for ax in axes.flat[n_cfg:]:
    ax.set_visible(False)

# outer labels
fig.text(0.5, 0.04, 'GameObject Z position', ha='center', fontsize=11)
fig.text(0.04, 0.5, 'Smoothed speed (m s⁻¹)', va='center',
         rotation='vertical', fontsize=11)
fig.suptitle('Average normalized speed vs Z (0.3 s smoothing; ±3.5 band)\nConfigs sorted by angle',
             y=0.98, fontsize=13)
fig.tight_layout(rect=[0.05, 0.05, 0.99, 0.95])
# ------------- add a single legend to the whole figure ---------------
fig.legend(handles=[bifurcation_line, goal_line],
           loc='upper right', bbox_to_anchor=(0.99, 0.99),
           frameon=False, fontsize=10)
plt.show()
